<a href="https://colab.research.google.com/github/ChristianFel7/Python-para-Data-Science/blob/main/Pandas_selecionando_e_agrupando_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

importando a base e selecionando os dados em que serão trabalhados




In [1]:
import pandas as pd


base disponível em: https://cdn3.gnarususercontent.com.br/2927-pandas-selecao-agrupamento-dados/1-SEEG10_GERAL-BR_UF_2022.10.27-FINAL-SITE.xlsx

In [2]:
emissoes_gases = pd.read_excel('/content/1-SEEG10_GERAL-BR_UF_2022.10.27-FINAL-SITE.xlsx', sheet_name = 'GEE Estados', engine='openpyxl')


FileNotFoundError: [Errno 2] No such file or directory: '/content/1-SEEG10_GERAL-BR_UF_2022.10.27-FINAL-SITE.xlsx'

In [ ]:
emissoes_gases.info()

In [ ]:
emissoes_gases = emissoes_gases[emissoes_gases['Emissão / Remoção / Bunker'] == 'Emissão']
emissoes_gases

In [ ]:

emissoes_gases = emissoes_gases.drop(columns = 'Emissão / Remoção / Bunker')
emissoes_gases

Agrupamento de dados

In [ ]:
colunas_info = list(emissoes_gases.loc[:, 'Nível 1 - Setor': 'Produto'].columns)
colunas_info

In [ ]:
colunas_emissao = list(emissoes_gases.loc[:, 1970: 2021].columns)
colunas_emissao

mudando o formato de long para wide

In [ ]:
emissoes_por_ano = emissoes_gases.melt(id_vars = colunas_info, value_vars = colunas_emissao, var_name = 'Ano', value_name = 'Emissão')
emissoes_por_ano

Agrupando pelo tipo de gas

In [ ]:
emissoes_por_ano.groupby('Gás').groups

In [ ]:
emissoes_por_ano.groupby('Gás').get_group('CO2 (t)')

In [ ]:
emissoes_por_ano.groupby('Gás').sum(numeric_only= True)

In [ ]:
emissao_por_gas = emissoes_por_ano.groupby('Gás').sum(numeric_only= True).sort_values('Emissão', ascending = False)
emissao_por_gas

In [ ]:
emissao_por_gas.plot(kind = 'barh', figsize = (10, 5))

In [ ]:
print(f'A emissão de CO2 corresponde a {float((emissao_por_gas.iloc[0:9].sum()/emissao_por_gas.sum()).iloc[0])*100:.2f} % de emissão total de gases estufa no Brasil de 1970 a 2021.')

Agrupamentos Multi-index

In [ ]:
gas_por_setor = emissoes_por_ano.groupby(['Gás', 'Nível 1 - Setor'])[['Emissão']].sum()
gas_por_setor

In [ ]:
gas_por_setor.xs('CO2 (t)', level = 'Gás')

In [ ]:
gas_por_setor.xs(('CO2 (t)', 'Mudança de Uso da Terra e Floresta'), level = [0,1])

In [ ]:
gas_por_setor.xs('CO2 (t)', level = 'Gás').max()

In [ ]:
gas_por_setor.xs('CO2 (t)', level = 0).idxmax()

In [ ]:
gas_por_setor.groupby(level = 0).idxmax()

In [ ]:
gas_por_setor.groupby(level = 0).max()

In [ ]:
valores_maximos = gas_por_setor.groupby(level = 0).max().values
tabela_sumarizada = gas_por_setor.groupby(level = 0).idxmax()
tabela_sumarizada.insert(1, 'Quantidade de emissão', valores_maximos)
tabela_sumarizada

In [ ]:
emissoes_por_ano.groupby('Ano')[['Emissão']].mean().plot(figsize = (10, 6));

In [ ]:
gas_por_setor.swaplevel(0, 1)

In [ ]:
gas_por_setor.swaplevel(0, 1).groupby(level = 0).idxmax()

In [ ]:
emissoes_por_ano

In [ ]:
emissoes_por_ano['Ano'] = pd.to_numeric(emissoes_por_ano['Ano'])
emissoes_por_ano.groupby('Ano')['Emissão'].mean().plot(figsize = (10,6));

In [ ]:
emissoes_por_ano.groupby('Ano').mean().plot(figsize = (10,6));

In [ ]:
emissoes_por_ano.groupby(['Ano', 'Gás'])['Emissão'].mean()

In [ ]:
media_emissao_anual = emissoes_por_ano.groupby(['Ano', 'Gás']).mean().reset_index()
media_emissao_anual

In [ ]:
media_emissao_anual = media_emissao_anual.pivot_table(index = 'Ano', columns = 'Gás', values = 'Emissão')
media_emissao_anual

In [ ]:
media_emissao_anual.plot(subplots = True, figsize = (10,40));

Parte final - utilizando uma nova base do censo de 2022 do ibge (https://cdn3.gnarususercontent.com.br/2927-pandas-selecao-agrupamento-dados/POP2022_Municipios.xls)

In [ ]:
import pandas as pd
#url = 'https://cdn3.gnarususercontent.com.br/2927-pandas-selecao-agrupamento-dados/POP2022_Municipios.xls'
populacao_estados = pd.read_excel('/content/POP2022_Municipios.xls', sheet_name = 'Municípios', header= 1, skipfooter=34)
populacao_estados

definindo a emissao per capita

In [ ]:
populacao_estados.groupby('UF').sum(numeric_only= True)

In [ ]:
populacao_estados['POPULAÇÃO'].astype(int)

In [ ]:
populacao_estados[populacao_estados['POPULAÇÃO'].str.contains('\(', na=False)]

Tratando os dados

In [ ]:
populacao_estados = populacao_estados.assign(populacao_sem_parenteses = populacao_estados['POPULAÇÃO'].replace('\(\d{1,2}\)', '', regex = True),
                                             populacao = lambda x : x.loc[:, 'populacao_sem_parenteses'].replace('\.', '', regex = True))

In [ ]:
populacao_estados[populacao_estados['POPULAÇÃO'].str.contains('\(', na = False)]

In [ ]:
populacao_estados = populacao_estados.astype({'populacao':'int64'})

In [ ]:
emissao_estados = emissoes_por_ano[emissoes_por_ano['Ano'] == 2021].groupby('Estado')[['Emissão']].sum().reset_index()
emissao_estados

In [ ]:
dados_agrupados = pd.merge(emissao_estados, populacao_estados, left_on = 'Estado', right_on = 'UF')
dados_agrupados

In [ ]:
dados_agrupados.plot(x = 'populacao', y= 'Emissão', kind = 'scatter', figsize=(8,6));

In [ ]:
import plotly.express as px

In [ ]:
px.scatter(dados_agrupados, x = 'populacao', y = 'Emissão', hover_name = 'Estado', text='Estado', opacity=0)

obtendo emissao per capita

In [ ]:
dados_agrupados = dados_agrupados.assign(emissao_per_capita = dados_agrupados['Emissão'] / dados_agrupados['populacao']).sort_values('emissao_per_capita', ascending = False)
dados_agrupados

In [ ]:
px.bar(data_frame = dados_agrupados, x = 'Estado', y = 'emissao_per_capita')

In [ ]:
px.scatter(data_frame = dados_agrupados, x = 'populacao', y = 'Emissão', text = 'Estado', size = 'emissao_per_capita')